Question 1.1:

In [1]:
import random
from math import log

#Constants for weather conditions and games
N = 200
CLOUDY = "C"
PLEASANT = "P"
HOT = "H"
BASKETBALL = "B"
SOCCER = "S"
COMPUTER_GAMES = "C"

def generate_init_weather(file_name):
    """
    Generate the initial weather and write it to the file.
    Randomly selects one of three weather conditions.
    """
    random_integer = random.randint(1, 3)
    
    with open(file_name, 'w') as weather_file:
        if random_integer == 1:
            weather_file.write(CLOUDY)
            curr_weather = CLOUDY
        elif random_integer == 2:
            weather_file.write(PLEASANT)
            curr_weather = PLEASANT
        else:  # random_integer == 3
            weather_file.write(HOT)
            curr_weather = HOT
    
    return curr_weather

def generate_game(game_str, curr_weather):
    """
    Generate a game based on the current weather and append it to game_str.
    """
    random_float = random.uniform(0, 1)

    if curr_weather == CLOUDY:
        if random_float < 0.7:
            game_str += BASKETBALL
        elif random_float < 0.9:  # 0.7 + 0.2
            game_str += SOCCER
        else:
            game_str += COMPUTER_GAMES
    elif curr_weather == PLEASANT:
        if random_float < 0.6:
            game_str += BASKETBALL
        elif random_float < 0.9:  # 0.6 + 0.3
            game_str += SOCCER
        else:
            game_str += COMPUTER_GAMES
    elif curr_weather == HOT:
        if random_float < 0.8:
            game_str += COMPUTER_GAMES
        elif random_float < 0.9:  # 0.8 + 0.1
            game_str += SOCCER
        else:
            game_str += BASKETBALL

    return game_str

def generate_weather(file_name, curr_weather):
    """
    Generate the next weather condition based on the current weather and write it to the file.
    """
    with open(file_name, 'a') as weather_file:
        random_float = random.uniform(0, 1)
        
        if curr_weather == CLOUDY:
            if random_float < 0.5:
                weather_file.write(CLOUDY)
            elif random_float < 0.8:  # 0.5 + 0.3
                weather_file.write(PLEASANT)
                curr_weather = PLEASANT
            else:
                weather_file.write(HOT)
                curr_weather = HOT
        elif curr_weather == PLEASANT:
            if random_float < 0.5:
                weather_file.write(PLEASANT)
            elif random_float < 0.8:  # 0.5 + 0.3
                weather_file.write(HOT)
                curr_weather = HOT
            else:
                weather_file.write(CLOUDY)
                curr_weather = CLOUDY
        elif curr_weather == HOT:
            if random_float < 0.4:
                weather_file.write(CLOUDY)
                curr_weather = CLOUDY
            elif random_float < 0.8:  # 0.4 + 0.4
                weather_file.write(HOT)
            else:
                weather_file.write(PLEASANT)
                curr_weather = PLEASANT
    return curr_weather


game_str = ""

#Generate initial weather and append corresponding game
curr_weather = generate_init_weather('WEATHER_OUT.txt')
game_str = generate_game(game_str, curr_weather)

#Generate weather and corresponding games for N-1 iterations
for i in range(1, N):
    curr_weather = generate_weather('WEATHER_OUT.txt', curr_weather)
    game_str = generate_game(game_str, curr_weather)

# Write the final game string to the output file
with open('Game_out.txt', 'w') as game_file:
    game_file.write(game_str)

Question 1.2:

In [2]:
def calc_v1(first_game, weather_type):
    """
    Calculate the initial probability (v1) based on the first game and initial weather type.
    Uses logarithms of probabilities to prevent underflow.
    """
    if weather_type == CLOUDY:
        if first_game == BASKETBALL:
            return log(1/3) + log(0.7)
        elif first_game == SOCCER:
            return log(1/3) + log(0.2)
        elif first_game == COMPUTER_GAMES:
            return log(1/3) + log(0.1)
    elif weather_type == PLEASANT:
        if first_game == BASKETBALL:
            return log(1/3) + log(0.6)
        elif first_game == SOCCER:
            return log(1/3) + log(0.3)
        elif first_game == COMPUTER_GAMES:
            return log(1/3) + log(0.1)
    elif weather_type == HOT:
        if first_game == BASKETBALL:
            return log(1/3) + log(0.1)
        elif first_game == SOCCER:
            return log(1/3) + log(0.1)
        elif first_game == COMPUTER_GAMES:
            return log(1/3) + log(0.8)

def calc_vi(prev_C, prev_P, prev_H, curr_game, weather_type):
    """
    Calculate the viterbi variable (vi) for the current weather type.
    Uses the previous vi values and the current game to compute the new vi value.
    Also returns the weather type that maximizes the probability for the current step.
    """
    if weather_type == CLOUDY:
        # Find the maximum weather transition probability
        max_weather = find_max_weather(prev_C + log(0.5), prev_P + log(0.2), prev_H + log(0.4))
        if curr_game == BASKETBALL:
            return log(0.7) + max(prev_C + log(0.5), prev_P + log(0.2), prev_H + log(0.4)), max_weather
        elif curr_game == SOCCER:
            return log(0.2) + max(prev_C + log(0.5), prev_P + log(0.2), prev_H + log(0.4)), max_weather
        elif curr_game == COMPUTER_GAMES:
            return log(0.1) + max(prev_C + log(0.5), prev_P + log(0.2), prev_H + log(0.4)), max_weather
    elif weather_type == PLEASANT:
        # Find the maximum weather transition probability
        max_weather = find_max_weather(prev_C + log(0.3), prev_P + log(0.5), prev_H + log(0.2))
        if curr_game == BASKETBALL:
            return log(0.6) + max(prev_C + log(0.3), prev_P + log(0.5), prev_H + log(0.2)), max_weather
        elif curr_game == SOCCER:
            return log(0.3) + max(prev_C + log(0.3), prev_P + log(0.5), prev_H + log(0.2)), max_weather
        elif curr_game == COMPUTER_GAMES:
            return log(0.1) + max(prev_C + log(0.3), prev_P + log(0.5), prev_H + log(0.2)), max_weather
    elif weather_type == HOT:
        # Find the maximum weather transition probability
        max_weather = find_max_weather(prev_C + log(0.2), prev_P + log(0.3), prev_H + log(0.4))
        if curr_game == BASKETBALL:
            return log(0.1) + max(prev_C + log(0.2), prev_P + log(0.3), prev_H + log(0.4)), max_weather
        elif curr_game == SOCCER:
            return log(0.1) + max(prev_C + log(0.2), prev_P + log(0.3), prev_H + log(0.4)), max_weather
        elif curr_game == COMPUTER_GAMES:
            return log(0.8) + max(prev_C + log(0.2), prev_P + log(0.3), prev_H + log(0.4)), max_weather

def find_max_weather(prev_C, prev_P, prev_H):
    """
    Determine the weather type with the maximum probability.
    """
    max_weather = max(prev_C, prev_P, prev_H)
    if max_weather == prev_C:
        return CLOUDY
    elif max_weather == prev_P:
        return PLEASANT
    else:
        return HOT

# Lists to store viterbi variables for each weather type
vi_C_list = list()
vi_P_list = list()
vi_H_list = list()

# Read the first game and calculate initial viterbi variables
with open('GAME_OUT.txt', 'r') as game_file:
    first_game = game_file.read(1)
vi_C_list.append(calc_v1(first_game, CLOUDY))
vi_P_list.append(calc_v1(first_game, PLEASANT))
vi_H_list.append(calc_v1(first_game, HOT))

# Lists to store the most probable previous weather type for each weather type
pi_C_list = list()
pi_P_list = list()
pi_H_list = list()

# Read the games and calculate the viterbi variables for each step
with open('GAME_OUT.txt', 'r') as game_file:
    for i in range(1, N):
        curr_game = game_file.read(1)
        new_v, max_weather = (calc_vi(vi_C_list[i-1], vi_P_list[i-1], vi_H_list[i-1], curr_game, CLOUDY))
        vi_C_list.append(new_v)
        pi_C_list.append(max_weather)
        new_v, max_weather = (calc_vi(vi_C_list[i-1], vi_P_list[i-1], vi_H_list[i-1], curr_game, PLEASANT))
        vi_P_list.append(new_v)
        pi_P_list.append(max_weather)
        new_v, max_weather = (calc_vi(vi_C_list[i-1], vi_P_list[i-1], vi_H_list[i-1], curr_game, HOT))
        vi_H_list.append(new_v)
        pi_H_list.append(max_weather)

#Calculate the most probable sequence of weather types 
hmm_list = list()
max_weather = find_max_weather(vi_C_list[199], vi_P_list[199], vi_H_list[199])
hmm_list.append(max_weather)
for i in range(197, -2, -1):
    if max_weather == CLOUDY:
        hmm_list.append(pi_C_list[i+1])
    elif max_weather == PLEASANT:
        hmm_list.append(pi_P_list[i+1])
    elif max_weather == HOT:
        hmm_list.append(pi_H_list[i+1])
reversed_hmm_list = list(reversed(hmm_list))
hmm_string = ''.join(reversed_hmm_list)

# Write the calculated weather sequence to the output file
with open('HMM_OUT.txt', 'w') as hmm_file:
    hmm_file.write(hmm_string)

Question 1.3:

In [3]:
with open('WEATHER_OUT.txt') as weather_file:
    weather_string = weather_file.read()


TP = 0
FP = 0
TN = 0
FN = 0
# Iterate over each character in the weather_string and hmm_string
for i in range(0, N):
    if weather_string[i] == HOT:
        if hmm_string[i] == HOT:
            # Both predicted and actual weather are HOT (True Positive)
            TP += 1
        else:
            # Actual weather is HOT, but predicted weather is not (False Negative)
            FN += 1
    else:  # weather_string[i] != HOT
        if hmm_string[i] == HOT:
            # Actual weather is not HOT, but predicted weather is HOT (False Positive)
            FP += 1
        else:
            # Both predicted and actual weather are not HOT (True Negative)
            TN += 1

print('TP = ', TP)
print('TN = ', TN)
print('FP = ', FP)
print('FN = ', FN)

TP =  23
TN =  100
FP =  42
FN =  35
